In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
url = "https://my.pitt.edu"
email = 'mfk31'
passw = 'nmmeam4e'

In [3]:
ids = pd.read_csv('/users/madke/documents/SBDL_ids.csv')

In [5]:
driver = webdriver.Chrome()
driver.get(url)

menu_btn = driver.find_element_by_id("headerMenuDropdown")
menu_btn.click()

signin_btn = driver.find_element_by_xpath("/html/body/div[1]/div[4]/section[1]/div/nav/div/div[2]/ul/li[2]/ul/li[1]/a")
signin_btn.click()

username = driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/div[1]/input")
username.send_keys(email)

password =  driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/div[2]/input")
password.send_keys(passw)

submit_btn = driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/button")
submit_btn.click()

CALL SELF FOR SECURITY

In [7]:
email_btn = driver.find_element_by_xpath("/html/body/div[1]/div[5]/div[3]/div/div[2]/div[4]/div[3]/div[1]/div[1]/div/a")
email_btn.click()

In [8]:
driver.switch_to.window(driver.window_handles[1])

In [9]:
#email_search = driver.find_element_by_class_name("5j-xjOS68WJ1zbIJWTFG7")
email_search_activate = driver.find_element_by_id("searchBoxId-Mail")
email_search_activate.click()

email_search = driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div[1]/div[2]/div/div/div/div/div[1]/div/div[2]/div/input")


In [16]:
search_btn = driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div[1]/div[2]/div/div/div/div/div[1]/button")

In [37]:
def return_name(email):
    email_search.send_keys(email)
    email_search_activate.click()
    time.sleep(1)
    results = driver.find_element_by_id("searchSuggestionsCallout")
    soup = BeautifulSoup(results.get_attribute('innerHTML'), 'html.parser')
    name = soup.find("div", {"id":"searchSuggestion-0" })['aria-label']
    
    email_search.send_keys(Keys.CONTROL + "a");
    email_search.send_keys(Keys.DELETE);
    
    return(name)

In [12]:
emails = ids.iloc[:, 0].values

In [38]:
names = []
for i in emails:
    print(return_name(i))
    names.append(return_name(i))

Butt, Muizz
Bacco, Katie M
Habib, Olivia
Brothers, Jordan R
Li, James T
O'Kane, Brendan
Regan, Seamus S
Malanowski, Anna Victoria
Han, Emmy
Simoncini, Julia
Puma, John A
Schrand, Ella
Herron, Margaret M
Chhoeung, Jasmine T
Santos, Sebastian A
Jiang, Danny
Erich, Ryan M
Nowozeniuk, Isabella J
Brennan, Shamus Patrick
Laglia, Ava L
Albert, Zachary N
Passarello, Jonathan W
Passarello, Jonathan W
Ramsey, Lucas
Ramfos, Ashley E
Knapp, Eric G
Vennetti, Elizabeth M
Wiemann, John
Calhoun, Jaden N
Taylor, Paige A
Wimer, Sierra R
Hanrahan, Colin M
Blackford, Anabel Clare
McFetridge, Abigail R
Nestler, Richelle J
Sillah, Muhammed
McNulty, Chase K
Berzon, Leo M
Franco, Gina M
Trauger, Caleigh S
Nayak, Krishna P
Kalasabail, Ankita B
Oliver, Hannah E
Cammon, Meghan T
Scholer, Heidi Noel
Kurtz, Nicole
Yialamas, Sophia A
Gratkowski, Ian J
McGirr, Conor
Csernica, Anna L
Grose, Elizabeth
Hood, Zach H
Boyle, Daniel J
Khondaker, Audrika T
Williams, Mickela J
Zhang, Kenny
Erka, Brendan M
Sifnakis, Thalia K


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"id","selector":"searchSuggestionsCallout"}
  (Session info: chrome=86.0.4240.183)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.19041 x86_64)
